#  Segmentação de Clientes com base em RFM 
Considerando dados de varejo da base de dados 'Online Retail Dataset, disponível em: http://archive.ics.uci.edu/ml/datasets/online+retail

## 1. Configuração do Notebook e Carregamento dos Dados

In [1]:
# Importando Bibliotecas
import pandas as pd
import datetime as dt

In [2]:
# carregar dados 
retail_data = pd.read_excel('Online_Retail.xlsx')
retail_data.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Verificar número de linhas e colunas do dataframe
retail_data.shape

(541909, 8)

## 2. Limpeza dos dados

### 2.1. Removendo Registros Duplicados

In [4]:
# Excluir possíveis registros duplicados
retail_data = retail_data.drop_duplicates()
print(f'Número de linhas e colunas após a remoção de duplicados = {retail_data.shape}')


Número de linhas e colunas após a remoção de duplicados = (536641, 8)


### 2.2. Conhecendo os Dados

In [5]:
# Verificando um resumo das informações dos dados
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      536641 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.8+ MB


In [6]:
# Conhecendo as métricas iniciais das variáveis quantitativas
retail_data.describe().iloc[:,0:2]

,Quantity,UnitPrice
count,536641.000000,536641.000000
mean,9.620029,4.632656
std,219.130156,97.233118
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


**Tamanho do Dataframe:** O banco de dados possui 8 variáveis e 536.641 registros não duplicados.

**Dados faltantes:** Existem valores faltantes nas variáveis: Description e CustomerID


**Tipos de dados das variáveis:**
* São variáveis qualitativas nominais: InvoiceNo, StockCode, Description, CustumerID* e Country
* São variáveis qualitativas ordinais: InvoiceDate
* São variáveis quantitativas: Quantity e UnitPrice


**Dados aparentemente anômalos**: Nota-se a existência de valores mínimos negativos nas colunas Quantity e UnitPrice, o que não condiz com o esperado para estas variáveis neste problema de negócio. Portanto iremos investigar um pouco estas ocorrências de registros. Nossa hipótese é a de que os valores negativos possam se tratar de devoluções de produtos ou operações similares.

### 2.3. Investigando Registros com Quantidades e Valores Unitários Negativos

In [7]:
# quantidades e preços negativos aparecem nos mesmos registros?

retail_data[(retail_data.Quantity<0) & (retail_data.UnitPrice<0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


Nota-se que valores negativos não ocorrem concomitantemente entre quantidade de unidades e preços unitários, então devem se trarar de tipos de operações distintas. 

In [8]:
# Quantos registros apresentam preço unitário negativo

retail_data[retail_data.UnitPrice<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


Duas ocorrências apresentam preço unitário negativo, e elas estão ligadas a ajustes de crédito da carteira de devedores.

In [9]:
# Em quantos registros aparecem quantidades negativas?
retail_data[retail_data.Quantity<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


**10.587 registros apresentam valores negativos na quantidade**

Vamos selecionar um cliente listado entre os que possuem valores negativos de quantidade e observar seus registros:

In [10]:
retail_data[retail_data.CustomerID== 16446.0].sort_values(by='InvoiceDate')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
194354,553573,22980,PANTRY SCRUBBING BRUSH,1,2011-05-18 09:52:00,1.65,16446.0,United Kingdom
194355,553573,22982,PANTRY PASTRY BRUSH,1,2011-05-18 09:52:00,1.25,16446.0,United Kingdom
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom


A partir da observação dos compras do cliente 17548	podemos confirmar a hipótese de que os valores negativos na variável Quantity correspondem a devoluções de mercadoria ou ações semelhantes.

### 2.4. Filtrando Registros de Acordo com o Escopo da Análise RFM

Uma vez que os dados de varejo estão sendo analisados com o objetivo de implementar uma análise de segmentação de clientes RFM (Recency, Frequency and Monetary Value), iremos aplicar alguns filtros para definir subconjuntos de dados em que o cliente é identificado e os registros de compras foram efetivos (ou seja, sem devoluções).

In [11]:
# Quantificando quantos registros em branco existem em cada dimensão
print (retail_data.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64


In [12]:
# Dataframe apenas com clientes identificados
retail_client =  retail_data[pd.notnull(retail_data['CustomerID'])].copy()
retail_client.shape

(401604, 8)

Para gerar um grupo de registros que contém apenas os casos de venta efetiva (sem devoluções) precisaremos buscar e excluir registros que apresentam duplicação nas variáveis StockCode, Description, UnitPrice e CustomerID

In [13]:
# Eliminando as vendas que não se concretizaram
mascara = retail_client[['StockCode', 'Description', 'UnitPrice', 'CustomerID']].duplicated(keep=False)
retail_client_sale =  retail_client[~mascara]

print(f'Número de linhas e colunas do dataframe com identificação de clientes e sem itens devolvidos = {retail_client_sale.shape}')

Número de linhas e colunas do dataframe com identificação de clientes e sem itens devolvidos = (213167, 8)


In [14]:
# Verirficando no dataframe retail_client_sale a situação do cliente 16446.0
retail_client_sale[retail_client_sale.CustomerID == 16446.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
194354,553573,22980,PANTRY SCRUBBING BRUSH,1,2011-05-18 09:52:00,1.65,16446.0,United Kingdom
194355,553573,22982,PANTRY PASTRY BRUSH,1,2011-05-18 09:52:00,1.25,16446.0,United Kingdom


Ao comparar os registros de compra do cliente 16446.0 apresentados acima com os apresentados no item 2.3, podemos notar que foram excluídos os registros referentes a compras que resultaram em devoluções.

In [15]:
# Verificar o resumo descritivo das métricas quantitativas
retail_client_sale.describe().iloc[:,0:2]

,Quantity,UnitPrice
count,213167.000000,213167.000000
mean,11.024403,3.420003
std,47.335779,88.125803
min,-9360.000000,0.000000
25%,2.000000,0.850000
50%,6.000000,1.690000
75%,12.000000,3.750000
max,12540.000000,38970.000000


Verificando o resumo descritivo do dataframe que possui identificação dos clientes e deveria ter apenas as vendas efetivas, verificamos que ainda existem quantidades negativas. Estas podem corresponder a devoluções de compras não registradas no banco de dados, conceção de descontos, brindes e benefícios a clientes.

**Aplicaremos um novo filtro para considerar apenas os registros com Quantity positivo**

In [16]:
retail_client_sale = retail_client_sale[retail_client_sale.Quantity >=0]
retail_client_sale.describe()

,Quantity,UnitPrice,CustomerID
count,211946.000000,211946.000000,211946.000000
mean,11.199155,3.015089,15287.584772
std,42.550645,12.780545,1711.293600
min,1.000000,0.000000,12347.000000
25%,2.000000,0.850000,13850.000000
50%,6.000000,1.690000,15253.000000
75%,12.000000,3.750000,16788.000000
max,12540.000000,2500.000000,18287.000000


Existem 211.946 registros que se tratam de itens de compras em que os clientes estão identificados.
Estes são os registros que iremos utilizar para a classificação dos clientes


## 3. Manipulação e Engenharia de dados

### 3.1. Trabalhando os dados e criando novas colunas

In [17]:
retail_client_sale.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
13,536367,22310,IVORY KNITTED MUG COSY,6,2010-12-01 08:34:00,1.65,13047.0,United Kingdom
19,536367,21777,RECIPE BOX WITH METAL HEART,4,2010-12-01 08:34:00,7.95,13047.0,United Kingdom
22,536368,22913,RED COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom
23,536368,22912,YELLOW COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom


In [18]:
# Adicionar Variável com o preço total por registro (Quantity x UnitPrice)
retail_client_sale['TotalPrice']=retail_client_sale.Quantity * retail_client_sale.UnitPrice

In [19]:
# Extraindo a data da informação de data e hora da coluna InvoiceDate
retail_client_sale['InvoiceDay'] = retail_client_sale.InvoiceDate.apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [20]:
retail_client_sale.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceDay
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08,2010-12-01
13,536367,22310,IVORY KNITTED MUG COSY,6,2010-12-01 08:34:00,1.65,13047.0,United Kingdom,9.90,2010-12-01
19,536367,21777,RECIPE BOX WITH METAL HEART,4,2010-12-01 08:34:00,7.95,13047.0,United Kingdom,31.80,2010-12-01
22,536368,22913,RED COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85,2010-12-01
23,536368,22912,YELLOW COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85,2010-12-01


### *** Definindo marco de data para análise

Precisamos definir um marco de tempo para a realização das análises, uma vez que estamos usando um conjunto de dados de exemplo datado entre 2010 e 2011, e não dados recentes.

Iremos definir como marco de data para consideração das análises o dia posterior ao mais recente do conjunto de dados, ou seja, dia mais recente + 1 dia.

In [21]:
pin_date = max(retail_client_sale.InvoiceDay) + dt.timedelta(1)

In [22]:
print(f'Data de referência para a análise {pin_date}')

Data de referência para a análise 2011-12-10 00:00:00


## 4. Aplicando Análise RFM

Agora iremos calcular as três métricas: Recency, Frequency e Monetary Value

**Recency** será dada pela diferença entre o marco de data de análise e a compra mais recente dos clientes

**Frequency** será dada pela contagem do número de compras (contabilizando a coluna InvoiceNo)

**Monetary Value** será dado pela soma dos valores totais de cada registro. 

In [23]:
# Calcular valores RFM
rfm = retail_client_sale.groupby('CustomerID').agg(Recency = ('InvoiceDate', lambda x: (pin_date - x.max()).days),
Frequency = ('InvoiceNo', 'count'), Monetary = ('TotalPrice', 'sum')).reset_index()
    
rfm

,CustomerID,Recency,Frequency,Monetary
0,12347.0,2,64,1597.03
1,12348.0,248,23,947.24
2,12349.0,18,73,1757.55
3,12350.0,310,17,334.40
4,12352.0,36,44,762.88
...,...,...,...,...
4294,18280.0,277,10,180.60
4295,18281.0,180,7,80.82
4296,18282.0,7,11,164.13
4297,18283.0,3,145,430.01


### 4.1. Agrupando RFM com base em sua distribuição em quartis

In [24]:
# Recency
rfm['RecencyClass'] =  pd.qcut(rfm.Recency, q=4, labels = [4, 3, 2, 1])

Lembrando que a métrica Recency indica o intervalo entre a data de análise e a última compra do cliente. Portanto, quanto menor este valor, maior é o engajamento do cliente, e maior sua pontuação de classificação.

In [25]:
# Frequency
rfm['FrequencyClass'] = pd.qcut(rfm.Frequency, q=4, labels=[1, 2, 3, 4])

In [26]:
# Monetary Value
rfm['MonetaryClass'] = pd.qcut(rfm.Monetary, q=4, labels = [1, 2, 3, 4])

In [27]:
# Definição de código para classe RFM geral
rfm['RFM_Class'] = rfm.apply(lambda x: str(x.RecencyClass)[0] + str(x.FrequencyClass)[0] + str(x.MonetaryClass)[0], axis=1)

# Calcular pontuação FRM
rfm['RFM_Score'] = rfm.apply(lambda x: int(x.RecencyClass) + int(x.FrequencyClass) + int(x.MonetaryClass), axis=1)

In [28]:
rfm

,CustomerID,Recency,Frequency,Monetary,RecencyClass,FrequencyClass,MonetaryClass,RFM_Class,RFM_Score
0,12347.0,2,64,1597.03,4,3,4,434,11
1,12348.0,248,23,947.24,1,2,3,123,6
2,12349.0,18,73,1757.55,4,4,4,444,12
3,12350.0,310,17,334.40,1,2,2,122,5
4,12352.0,36,44,762.88,3,3,3,333,9
...,...,...,...,...,...,...,...,...,...
4294,18280.0,277,10,180.60,1,1,1,111,3
4295,18281.0,180,7,80.82,1,1,1,111,3
4296,18282.0,7,11,164.13,4,1,1,411,6
4297,18283.0,3,145,430.01,4,4,2,442,10


### Classificando os clientes (Bronze, Silvel, Golden) de acorco om o RFM Score

In [29]:
rfm['Client_Level'] = pd.qcut(rfm.RFM_Score, q=3, labels=['Bronze', 'Silver', 'Gold'])

In [30]:
rfm.groupby('Client_Level').agg({'Recency':  ['min', 'max', 'median'], 'Frequency': ['min', 'max', 'median'], 
                                 'Monetary':['min', 'max', 'median']})

Recency             Frequency             Monetary            \
                 min  max median       min  max median      min       max   
Client_Level                                                                
Bronze             0  373  163.0         1   68     12     0.00  18311.50   
Silver             0  373   45.5         1  341     37    89.94  12393.70   
Gold               0  148   15.0        18  937     88   280.06  67466.01   

                       
               median  
Client_Level           
Bronze         219.52  
Silver         560.01  
Gold          1262.01

Com esta classificação podemos identificar desde os clientes mais fiéis até aqueles mais distantes. Assim, estratégias de marketing podem ser feitas considerando os diferentes grupos de clientes.

De acordo com as especificidades de cada negócio podem ser definidas diferentes níveis e classes de clientes, e análises complementares podem ser feitas para entender melhor o perfil de compra de cada uma destas classes (quais são os itens preferidos, se há padrões de dias e horários preferidos para as compras, etc.)